In [1]:
import numpy as np
import math as mt
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
from scipy import signal
import IPython
import warnings
from pylab import *
warnings.filterwarnings("ignore")
%matplotlib inline

# Testiamo i filtri su suoni complessi!

### Low End Head Bumps
+ Achieved via a peak filter:

    + DARE NOMI CORRETTI A VARIABILI

$$HB(x) = 1 + \frac{H_0}{2} \cdot [1 - A_2(x)]$$

+ Where A2 is

$$A_2(x) = \cfrac{-\frac{CB}{C} + d \cdot ( 1 -\frac{CB}{C}) \cdot x^{-1} + x^{-2}}{ 1 + d \cdot ( 1 -\frac{CB}{C}) \cdot x^{-1} - \frac {CB}{Cx^{-2}}}$$

In [2]:
def head_bumps(track, fs, fc, bw, G): 
    # normalized center frequency
    # reccomended between 45-100Hz
    Wc = (2*fc)/fs

    # normalized bandwidth
    # reccomended between 50-200Hz
    Wb = (2*bw)/fs
    
    V0 = mt.pow(10, (G/20))
    H0 = V0 - 1

    if (G >= 0):
        # boost
        c = (np.tan((np.pi*Wb)/2)-1) / (np.tan((np.pi*Wb)/2)+1) 
    else:
        # cut
        c = (np.tan((np.pi*Wb)/2)-V0) / (np.tan((np.pi*Wb)/2)+V0) 

    d = -np.cos(np.pi*Wc)

    track_h = [0, 0]
    out = np.zeros_like(track)

    for n in range(1, len(track)):
        track_h_new = track[n] - d*(1-c)*track_h[0] + c*track_h[1]
        ap_y = -c * track_h_new + d*(1-c)*track_h[0] + track_h[1]
        track_h = [track_h_new, track_h[0]]
        out[n] = 0.5 * H0 * (track[n] - ap_y) + track[n]
    return out

### Tape Saturation
+ Achieved via a ArcTan filter:

$$ TP(x) = 2\pi \arctan(\alpha \cdot x)$$

In [3]:
def tape_sat(track, alpha):
    out = np.zeros_like(track)

    for n in range (1, len(track)):
        out[n] = 2 * np.pi * np.arctan(alpha * track[n]) 
    return out

### Wows and Flutters
+ Achieved via a Vibrato filter

In [4]:
def vibrato(track, samplerate, modfreq, width):
    # Normalized modulation width
    width_samples = round(width * samplerate)
    
    # Normalized modulation frequency
    modfreq_samples = modfreq / samplerate
    
    # Number of samples in sound
    length_track = len(track)
    
    # Length of the entire delay
    L = 2 + width_samples * 3
    
    # Memory allocation for delay
    delay_line = np.zeros(L)
    
    # Memory allocation for output vector
    y = np.zeros_like(track)

    ya_alt=0
    
    for n in range(1, length_track):
        mod = np.sin(modfreq_samples * 2 * np.pi * n)
        tap = 1 + width_samples * 2 * mod
        i = int(np.floor(tap))
        frac = tap - i
        
        delay_line = np.concatenate(([track[n]], delay_line[:L-1]))

        # Linear Interpolation
        y[n] = delay_line[i+1] * frac + delay_line[i] * (1 - frac)

        # Allpass Interpolation 
        # y[n] = delay_line[i + 1] + (1 - frac) * delay_line[i] - (1 - frac) * ya_alt
        # ya_alt = y[n]
        
        # Spline Interpolation 
        # y[n] = (delay_line[i + 1] * frac**3 / 6
        #         + delay_line[i] * ((1 + frac)**3 - 4 * frac**3) / 6
        #         + delay_line[i - 1] * ((2 - frac)**3 - 4 * (1 - frac)**3) / 6
        #         + delay_line[i - 2] * (1 - frac)**3 / 6)
    
    return y